# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`
mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray



In [49]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [50]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [51]:
# confirm that our new database was created
mongo.list_database_names()


['admin',
 'auto',
 'autosaurus',
 'classDB',
 'config',
 'fruits_db',
 'local',
 'uk_food']

In [52]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [53]:
# review the collections in our new database
db.list_collection_names()


['establishments']

In [54]:
# review a document in the establishments collection
establishments = db['establishments']
document = establishments.find_one()
pprint(document)


{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64b04e7e82bb94ffe1427168'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [55]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [56]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [57]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant)


In [58]:
# Check that the new restaurant was inserted
pprint(new_restaurant)



{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64b051e9288e06275207cfb1'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [74]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
result = establishments.find(query, projection).limit(1)
for document in result:
    pprint(document)


{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [75]:
# Update the new restaurant with the correct BusinessTypeID
query = {"name": "Penang Flavours"}
update = {"$set": {"BusinessTypeID": 1}}
establishments.update_one(query, update)

In [78]:
# Confirm that the new restaurant was updated
new_restaurant = establishments.find_one(query)

pprint(new_restaurant)


{'BusinessTypeID': 1,
 '_id': ObjectId('64b0500b288e06275207cfac'),
 'address': 'Greenwich',
 'name': 'Penang Flavours',
 'rating': None}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [79]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}
count = establishments.count_documents(query)

print(f"Number of documents with LocalAuthorityName as 'Dover': {count}")

Number of documents with LocalAuthorityName as 'Dover': 994


In [80]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {"LocalAuthorityName": "Dover"}
result = establishments.delete_many(query)

print(f"Number of documents deleted: {result.deleted_count}")

Number of documents deleted: 994


In [81]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName": "Dover"}
remaining_documents = establishments.count_documents(query)

if remaining_documents > 0:
    print("There are remaining documents that include 'Dover'")
else:
    print("No remaining documents include 'Dover'")

No remaining documents include 'Dover'


In [82]:
# Check that other documents remain with 'find_one'
query = {"LocalAuthorityName": {"$ne": "Dover"}}
remaining_document = establishments.find_one(query)

if remaining_document:
    pprint(remaining_document)
else:
    print("No remaining documents found")

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64b04e7e82bb94ffe1427430'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [84]:
# Change the data type from String to Decimal for longitude and latitude
# Retrieve the documents that have string values for longitudeand latitude
query = {"$or": [ 
    {"longitude": {"$type": "string"}},
    {"latitude": {"$type": "string"}}
]}
documents_to_update = establishments.find(query)

# Update the data type for longitude and latitude fields
for document in documents_to_update:
    longitude = Decimal(document['longitude'])
    latitude = Decimal(document['latitude'])
    establishments.update_one({"_id": document['_id']}, {"$set": {"longitude": longitude, "latitude": latitude}})

print("Data type updated successfully for longitude and latitude fields.")

Data type updated successfully for longitude and latitude fields.


Use `update_many` to convert `RatingValue` to integer numbers.

In [85]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [86]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({"RatingValue": {"$type": "string"}}, [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}])


In [87]:
# Check that the coordinates and rating value are now numbers
# Check coordinates
coordinates_query = {"$or": [
    {"longitude": {"$type": "decimal"}},
    {"latitude": {"$type": "decimal"}}
]}
coordinates_documents = establishments.find(coordinates_query)
if coordinates_documents.count() == 0:
    print("Coordinates have been successfully updated to numbers.")
else:
    print("Coordinates still contain non-number values.")

# Check rating value
rating_query = {"RatingValue": {"$type": "int"}}
rating_documents = establishments.find(rating_query)
if rating_documents.count() > 0:
    print("Rating value has been successfully updated to numbers.")
else:
    print("Rating value still contains non-number values.")

Coordinates have been successfully updated to numbers.
Rating value has been successfully updated to numbers.


C:\Users\binhd\AppData\Local\Temp\ipykernel_26744\3001195531.py:8: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  if coordinates_documents.count() == 0:
C:\Users\binhd\AppData\Local\Temp\ipykernel_26744\3001195531.py:16: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  if rating_documents.count() > 0:
